In [337]:
import string
import numpy as np
import pandas as pd
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as pgs

# Esercizio 2.2 - Topic modeling

Lo scopo di questo esercizio è quello di ottenere gli argomenti trattati (__topics__) a partire da un corpus.
Il corpus è un mix di documenti riguardanti _sport_, _cibo_ e _politica_, abbiamo dunque 3 topics.
La ripartizione dei documenti fra i vari topics è:
- 4 per lo _sport_
- 6 per la _politica_
- 5 per il _cibo_

in totale abbiamo 15 documenti.

Carichiamo il corpus in un dataframe pandas, ogni riga corrisponde alle frasi di uno dei documenti della cartella _lda_sports_politics_food_.

In [338]:
files_name = ['badminton.txt', 'barack obama.txt', 'baseball.txt', 'cricket.txt', 'dosa.txt', 'idli.txt', 'lee quan yew.txt', 'narendra modi.txt', 'noodles.txt', 'pasta.txt', 'pizza.txt', 'queen elizabeth.txt', 'shinzo abe.txt', 'table tennis.txt', 'tipu sultan.txt']
docs = []
for file in files_name:
    docs.append(open('utils/lda_sports_politics_food//' + file, 'r').readlines()[0])
df = pd.DataFrame(data=docs[:], columns=['documents'])
df

,documents
0,Badminton is a racquet sport played using racq...
1,"Barack Hussein Obama II (born August 4, 1961) ..."
2,Baseball is a bat-and-ball game played between...
3,Cricket is a bat-and-ball game played between ...
4,Dosa is a kind of pancake made from a fermente...
5,Idli is a traditional breakfast in South India...
6,"Lee Kuan Yew, GCMG, CH, SPMJ (born Harry Lee K..."
7,Narendra Damodardas Modi (born 17 September 19...
8,Noodles are a staple food in many cultures mad...
9,Pasta is a staple food of traditional Italian ...


Effettuiamo preprocessing.

In [339]:
stopwords = []
for line in open("utils/stop_words_FULL.txt", 'r').readlines():
    stopwords.append(line.rstrip('\n'))
stopwords = np.array(stopwords)


def preprocessing(s):
    """
    Do some preprocessing operations on the string.

    :param s: the string

    :return: the preprocessed string
    """
    # Lowercasing
    s = s.lower()
    # Punct removal
    s = s.translate(str.maketrans('', '', string.punctuation))
    # Stopword removal
    s = ' '.join([word for word in s.split() if word not in stopwords])
    return s

In [340]:
vect_preprocessing = np.vectorize(preprocessing) # vettorizziamo la funzione in modo da applicarla a tutto l'array di frasi in maniera efficiente
df['documents'] = vect_preprocessing(df['documents'])
df['documents'] = df['documents'].apply(lambda x: x.split()) # dividiamo le frasi in liste di parole così da averle già pronte per il modello
df

,documents
0,"[badminton, racquet, sport, played, racquets, ..."
1,"[barack, hussein, obama, ii, born, august, 4, ..."
2,"[baseball, batandball, game, played, teams, pl..."
3,"[cricket, batandball, game, played, teams, ele..."
4,"[dosa, kind, pancake, fermented, batter, main,..."
5,"[idli, traditional, breakfast, south, indian, ..."
6,"[lee, kuan, yew, gcmg, ch, spmj, born, harry, ..."
7,"[narendra, damodardas, modi, born, 17, septemb..."
8,"[noodles, staple, food, cultures, unleavened, ..."
9,"[pasta, staple, food, traditional, italian, cu..."


Costruiamo il modello per il task di __topic modeling__, scegliamo quello utilizzante la _Latent Dirichlet Allocation_, sistema basato sulla statistica bayesiana in cui ogni parola ha una certa probabilità di comparire in un singolo topic.

In [344]:
# creiamo un dizionario a partire dalla lista di parole ottenuta dai documenti del corpus
# la classe Dictionary() di gensim genera un dizionario inizializzato con le parole
dictionary = Dictionary(df['documents'])

# dalle parole passiamo ai valori delle loro occorrenze nel testo
corpus = [dictionary.doc2bow(text) for text in df['documents']] # lista di coppie (id parola, numero di occorrenze)

# alleniamo il modello
lda_model = LdaModel(corpus=corpus, # le occorrenze
                     id2word=dictionary, # le parole
                     num_topics=3, # il numero di topics che vogliamo ottenere
                     random_state=27, # per la riproducibilità degli esperimenti
                     update_every=1, # numero di documenti per ogni iterazione prima di aggiornare il modello
                     chunksize=1, # numero di documenti da usare ad ogni iterazione
                     alpha="auto") # credenza a priori sulla distribuzione dei topic, con "auto" la impara da solo

lda_model.show_topics() # abbiamo una lista di parole con le rispettive probabilità di appartenere a quel topic

[(0,
  '0.020*"ball" + 0.020*"table" + 0.014*"pizza" + 0.011*"return" + 0.011*"players" + 0.010*"side" + 0.010*"rules" + 0.007*"fast" + 0.007*"generally" + 0.007*"serve"'),
 (1,
  '0.031*"abe" + 0.019*"minister" + 0.018*"prime" + 0.014*"father" + 0.014*"war" + 0.012*"september" + 0.011*"born" + 0.010*"kingdom" + 0.009*"election" + 0.009*"elizabeth"'),
 (2,
  '0.033*"tipu" + 0.019*"mysore" + 0.015*"sultan" + 0.011*"ali" + 0.011*"french" + 0.011*"british" + 0.011*"pasta" + 0.009*"shapes" + 0.008*"second" + 0.008*"anglomysore"')]

## Visualizzazione dei risultati

Per valutare la bontà del modello usiamo due misure:
- la __perplexity__, fornisce informazione su quanto il modello è incerto nel processare nuovi dati. Viene calcolata con la log-likelihood sul test set.
- la __choerence__, misura il grado di coerenza fra le parole di un topic. Viene calcolata guardando alla similarità semantica fra le parole con la più alta probabilità del topic

In [342]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['documents'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


Perplexity:  -7.555826722110876
Coherence Score:  0.6058535509700912


Per la visualizzazione dei risultati è stata usata la libreria __pyLDAvis__ (<https://github.com/bmabey/pyLDAvis>) che fornisce una comoda interfaccia interattiva per la visualizzazione dei cluster e delle loro parole rilevanti.

In [343]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pgs.prepare(lda_model, corpus, dictionary)
vis

/home/pietro/Scrivania/python venv/venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.090301  0.098243       1        1  36.862922
2     -0.150210  0.014210       2        1  32.902918
0      0.059909 -0.112454       3        1  30.234160, topic_info=           Term       Freq      Total Category  logprob  loglift
863        tipu  16.000000  16.000000  Default  30.0000  30.0000
697         abe  16.000000  16.000000  Default  29.0000  29.0000
138        ball   8.000000   8.000000  Default  28.0000  28.0000
770       table   8.000000   8.000000  Default  27.0000  27.0000
354    minister  10.000000  10.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
177   opponents   2.207372   2.571959   Topic3  -5.1974   1.0433
182        play   2.207372   2.571959   Topic3  -5.1974   1.0433
595  neapolitan   2.244851   2.921575   Topic3  -5.1806   0.9327
521     divided   2.179418   2.933068   Topic3  -5.2101   0.8992
443        time   2.178500   3.796582   Topic3  -5.2106   0.6407

[139 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
622       1  1.034397    12
774       2  0.861641  1750
775       1  0.245221  1782
775       2  0.735664  1782
776       2  0.861641  1784
...     ...       ...   ...
443       1  0.263395  time
443       3  0.526790  time
863       1  0.122134  tipu
863       2  0.854936  tipu
690       1  0.951372   war

[135 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

Sono state effettuate diverse esecuzioni cambiando il numero di topics fra i parametri del modello.

Come prima esecuzione è stato impostato __num_topics=3__ in quanto siamo a conoscenza dell'esistenza di 3 topic nel dataset.
Il sistema riesce a riconoscere il cluster riguardante la _politica_ e quello riguardante gli _sport_ ma non sembra in grado di riconoscere quello sul _cibo_, mischiando parole dagli altri due cluster.

Provando ad incrementare il numero di topics a __num_topics=5__ notiamo subito un miglioramento delle misure di valutazione ed anche i clusters dei topics sono più precisi.
Il modello individua chiaramente i tre topics su _politica_, _cibo_ e _sport_, emerge inoltre un quarto topic in cui sono racchiuse parole relative alla _guerra_ mentre l'ultimo non sembra avere un significato preciso.
